In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

# Audio.Utils
Resourses and useful links:

 - ...

In [ ]:
#|default_exp audio.utils
#|export
import librosa
from resemblyzer.hparams import audio_norm_target_dBFS
import numpy as np
from pathlib import Path
int16_max = (2 ** 15) - 1

#!
#! Depreciated
#!
def normalize_volume(wav, target_dBFS, increase_only=False, decrease_only=False):
    if increase_only and decrease_only:
        raise ValueError("Both increase only and decrease only are set")
    rms         = np.sqrt(np.mean((wav * int16_max) ** 2))
    wave_dBFS   = 20 * np.log10(rms / int16_max)
    dBFS_change = target_dBFS - wave_dBFS
    if dBFS_change < 0 and increase_only or dBFS_change > 0 and decrease_only:
        return wav
    
    return wav * (10 ** (dBFS_change / 20))

#!
#! Depreciated
#!
def preprocess_wav(fpath_or_wav: Path):
    # Load the wav from disk if needed
    wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
        
    # Apply the preprocessing: normalize volume and shorten long silences 
    wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)
    
    return wav, source_sr

In [ ]:
#|export
from resemblyzer import VoiceEncoder

def compute_embedding(wav, rate=1.3):
    """
    Cuts up wav in chunks of width 25600 = 1.6*16_000
    """
    enc = VoiceEncoder("cpu")
    #      
    # rate: how many partial utterances should occur per second
    # len(X) ~ len(wav)/(16_000/rate)
    _, X, S = enc.embed_utterance(wav[:], return_partials=True, rate=rate) 
    return X,S


In [ ]:
#|hide
rate = 11 # restarts per second

wav = np.random.rand(16_000*20)
x, s = compute_embedding(wav, rate=rate)

#
#  Understanding where the slices come from
#  
step_ms = 10                  # fft-step in ms
step    = step_ms/1000*16_000 # fft-step in samples
res     = 16_000/rate         # restart in samples
res_fr  = res//step           # restart in frames/mel-columns
res_    = int(res_fr * step)  # restart in samples again, but rounding happened

assert res_ == s[1].start, (res,s[1].start)